# V2: Build a CNN with three VGG Blocks, with Drop out regularization

Import prerequisits

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from keras_preprocessing.image import ImageDataGenerator
import os, sys

Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Specify paths to train and test image sets

In [3]:
train_set_path = '/content/drive/MyDrive/Colab Notebooks/cats_and_dogs/train'
test_set_path = '/content/drive/MyDrive/Colab Notebooks/cats_and_dogs/test'

Following function will create a CNN with 3 VGG blocks

In [1]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

Function to evaluate Cross Entropy Loss and Classification Accuracy

In [5]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
  # plot loss
  plt.subplot(211)
  plt.title('Cross Entropy Loss')
  plt.plot(history.history['loss'], color='blue', label='train')
  plt.plot(history.history['val_loss'], color='orange', label='test')
  # plot accuracy
  plt.subplot(212)
  plt.title('Classification Accuracy')
  plt.plot(history.history['accuracy'], color='blue', label='train')
  plt.plot(history.history['val_accuracy'], color='orange', label='test')
  # save plot to file
  #filename = sys.argv[0].split('/')[-1]
  #plt.savefig(filename + '_plot.png')
  plt.show()
  plt.close()

Image Data Generator with rescaling images

In [6]:
# Data generator, same generator for train set and test set
data_gen = ImageDataGenerator(rescale=1.0/255.0)

train_gen = data_gen.flow_from_directory(train_set_path,
                                        class_mode='binary',
                                        batch_size=64,
                                        target_size=(200, 200))

test_gen = data_gen.flow_from_directory(test_set_path,
                                        class_mode='binary',
                                        batch_size=64,
                                        target_size=(200, 200))

Found 18697 images belonging to 2 classes.
Found 6301 images belonging to 2 classes.


Train Model

In [ ]:
# Train a model
model = define_model()

monitor = EarlyStopping(monitor='val_loss', 
                        min_delta=1e-3, 
                        patience=5, 
                        verbose=1, 
                        mode='auto',
                        restore_best_weights=True)

history = model.fit(train_gen, 
                    steps_per_epoch=len(train_gen),
                    validation_data=test_gen, 
                    validation_steps=len(test_gen), 
                    epochs=100, 
                    verbose=1,
                    callbacks=[monitor])

# Model accuracy on test set
accuracy_score = model.evaluate_generator(test_gen, steps=len(test_gen), verbose=0)

summarize_diagnostics(history)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
246/293 [========================>.....] - ETA: 35:55 - loss: 0.6889 - accuracy: 0.5844

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

293/293 [==============================] - ETA: 0s - loss: 0.6814 - accuracy: 0.5913 